In [ ]:
import numpy as np 
import torch
import os 
import nltk
import random
import torch.cuda as cuda
import pickle
from nltk.tokenize import word_tokenize
nltk.download('punkt')

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) # Show the GPU & CPU Specifications

from google.colab import drive
drive.mount('/content/drive')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6913916854135054574
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15505193728
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9101631424716308588
physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0"
]
Mounted at /content/drive


In [ ]:
### division ###

train_samples_path = "/content/drive/MyDrive/sentiment/train.ft.txt"

file = open(train_samples_path, "r")
train = []
for line in file.readlines():
  train.append( line )
file.close()

random.shuffle(train)

with open("/content/drive/MyDrive/sentiment/train_data.txt", "w") as output:
    output.writelines(train[:800000])
with open("/content/drive/MyDrive/sentiment/validation_data.txt", "w") as output:
    output.writelines(train[800000:1000000])    


In [ ]:
train_samples_path = "/content/drive/MyDrive/sentiment/train_data.txt"
validation_samples_path = "/content/drive/MyDrive/sentiment/validation_data.txt"
test_samples_path = "/content/drive/MyDrive/sentiment/test.ft.txt"

file = open(train_samples_path, "r")
train_data = []
train_label = []
for line in file.readlines():
  x = line[11:-1].lower().replace(".", " . ")   
  train_data.append( word_tokenize(x) )
  train_label.append(int(line[9]) - 1)
file.close()

file = open(validation_samples_path, "r")
validation_data = []
validation_label = []
for line in file.readlines():
  x = line[11:-1].lower().replace(".", " . ")   
  validation_data.append( word_tokenize(x) )
  validation_label.append(int(line[9]) - 1)
file.close()

file = open(test_samples_path, "r")
test_data = []
test_label = []
for line in file.readlines():
  x = line[11:-1].lower().replace(".", " . ")   
  test_data.append( word_tokenize(x) )
  test_label.append(int(line[9]) - 1)
file.close()

#### Train data is 2D array and train label is 1D array

print("Sample 0 of train data : {}".format(train_data[0]))
print("Sample 0 of train label : {}".format(train_label[0]))

#### Validation data is 2D array and train label is 1D array

print("Sample 0 of validation data : {}".format(validation_data[0]))
print("Sample 0 of validation label : {}".format(validation_label[0]))

#### Test data is 2D array and train label is 1D array

print("Sample 0 of test data : {}".format(test_data[0]))
print("Sample 0 of test label : {}".format(test_label[0]), end = "\n")


print("Lenght of train data is {}".format(len(train_data)))
print("Lenght of validation data is {}".format(len(validation_data)))
print("Lenght of test data is {}".format(len(test_data)))

Sample 0 of train data : ['by', 'itself', ',', 'the', 'buffer', 'ai', "n't", 'up', 'to', 'snuff', '.', 'get', 'the', 'full', 'kit', 'for', 'a', 'few', 'dollars', 'more', '.', ':', 'this', 'pedicure', 'foot', 'care', 'kit', 'is', 'ideal', 'for', 'people', 'who', 'have', 'slightly', 'calloused', 'soles', 'and', 'heels', '.', 'the', 'battery-powered', 'exfoliator', 'has', 'three', 'interchangeable', 'discs', 'with', 'different', 'textures', '.', 'the', 'discs', 'rotate', ',', 'much', 'like', 'an', 'electric', 'sander', '.', 'however', ',', 'because', 'it', 'is', 'powered', 'by', 'only', 'two', 'aa', 'batteries', '(', 'not', 'included', 'in', 'the', 'kit', ')', ',', 'too', 'much', 'pressure', 'causes', 'the', 'disc', 'to', 'stop', 'spinning', '.', 'this', 'may', 'not', 'be', 'a', 'flaw', 'in', 'its', 'design', ',', 'but', 'actually', 'a', 'safety', 'feature', '.', 'the', 'unit', 'is', 'very', 'light', 'and', 'fits', 'comfortably', 'in', 'your', 'hand', '.', 'it', 'is', 'also', 'covered', '

In [ ]:
vocab = ['UNK', 'PAD']
vocab_repeat = [0, 0] # Number of repetitions of vocabs

### Creating Vocabulary from training data 
for sample in train_data:
  for word in sample:
    if word not in vocab:
      vocab.append(word)
      vocab_repeat.append(1)
    else:
      index = vocab.index(word)
      vocab_repeat[index] += 1   


#### Vocabs with less than 10 repetitions will be replaced with UNK.
Minimum_of_repetition = 10
vocab_remove = []
for index_sample, sample in enumerate(train_data):
  for index_word, word in enumerate(sample):
    if  vocab_repeat[vocab.index(word)] < Minimum_of_repetition:
        train_data[index_sample][index_word] = 'UNK'
        if word not in vocab_remove:
          vocab_remove.append(word)

for word in vocab_remove:
  index = vocab.index(word)
  vocab.remove(word)
  del(vocab_repeat[index])

print("The size of the vocabulary is : {}".format(len(vocab)))
print("The vocab is : \n{}\n".format(vocab))
print("The vocab repeat is : \n{}\n".format(vocab_repeat))




The size of the vocabulary is : 64101
The vocab is : 
['UNK', 'PAD', 'by', 'itself', ',', 'the', 'buffer', 'ai', "n't", 'up', 'to', 'snuff', '.', 'get', 'full', 'kit', 'for', 'a', 'few', 'dollars', 'more', ':', 'this', 'pedicure', 'foot', 'care', 'is', 'ideal', 'people', 'who', 'have', 'slightly', 'soles', 'and', 'heels', 'battery-powered', 'has', 'three', 'interchangeable', 'discs', 'with', 'different', 'textures', 'rotate', 'much', 'like', 'an', 'electric', 'sander', 'however', 'because', 'it', 'powered', 'only', 'two', 'aa', 'batteries', '(', 'not', 'included', 'in', ')', 'too', 'pressure', 'causes', 'disc', 'stop', 'spinning', 'may', 'be', 'flaw', 'its', 'design', 'but', 'actually', 'safety', 'feature', 'unit', 'very', 'light', 'fits', 'comfortably', 'your', 'hand', 'also', 'covered', 'one-year', 'warranty', 'inspiring', 'i', 'first', 'followed', 'book', 'although', 'menu', 'that', 'great', 'when', 'you', "'re", 'trying', 'lean', 'there', 'appetizing', 'if', 'were', 'recipes', 'wou

In [ ]:
## Evaluate sizes - calculating MIN, MAX, MEAN‌, STD
mean_size_of_samples = 0
std_of_samples = 0  
max = 0
min =100000

for sample in train_data:
  mean_size_of_samples += len(sample)
  if len(sample) > max :
    max = len(sample)
  if len(sample) < min :
    min = len(sample)

mean_size_of_samples = mean_size_of_samples / len(train_data)

for sample in train_data:
  std_of_samples += (len(sample) - mean_size_of_samples)**2
std_of_samples = np.sqrt( std_of_samples / len(train_data) )

print("Train:  Max sample length is : {}".format(max))
print("Train: Min sample length is : {}".format(min))
print("Train: Mean sample length is : {}".format(mean_size_of_samples))
print("Train: Std of lenghts is : {}".format(std_of_samples))

sU = mean_size_of_samples + 2*std_of_samples

print("Mean + 2 * Std  is : {}".format(sU))


### Check : How many train samples have lenght <= SU
count = 0
for sample in train_data:
  if ( len(sample) <= int(sU)  ):
    count += 1
print("It is {}% of samples".format(count/len(train_data) * 100 ))


Train:  Max sample length is : 340
Train: Min sample length is : 13
Train: Mean sample length is : 92.789695
Train: Std of lenghts is : 50.22857151868215
Mean + 2 * Std  is : 193.24683803736428
It is 95.889625% of samples


In [ ]:
## ADD Padding and Cutting the long samples 
seq_lenght = int(sU) + 1
print("The sequence lenght is {}".format(seq_lenght))

## If the size of sample is less than seq_lenght, we add enough <pad> to reach the size of seq_lenght. 
## Or if the size of sample is bigger than seq_lenght, we just use the first seq_lenght vocabs. 

for idx_sample, sample in enumerate(train_data):
  if len(sample) < seq_lenght:
    for i in range( seq_lenght -len(sample) ):
      train_data[idx_sample].append('PAD')
  else :
      train_data[idx_sample] = train_data[idx_sample][0:seq_lenght]

for idx_sample, sample in enumerate(validation_data):
  if len(sample) < seq_lenght:
    for i in range( seq_lenght -len(sample) ):
      validation_data[idx_sample].append('PAD')
  else :
      validation_data[idx_sample] = validation_data[idx_sample][0:seq_lenght]

for idx_sample, sample in enumerate(test_data):
  if len(sample) < seq_lenght:
    for i in range( seq_lenght -len(sample) ):
      test_data[idx_sample].append('PAD')
  else :
      test_data[idx_sample] = test_data[idx_sample][0:seq_lenght]


The sequence lenght is 194


In [ ]:
## Convert Vocabs to their indexes in Vocab. 

for idx_sample, sample in enumerate(train_data):
  for idx_word, word in enumerate(sample):
    train_data[idx_sample][idx_word] = vocab.index(word)    


for idx_sample, sample in enumerate(validation_data):
  for idx_word, word in enumerate(sample):
    if word not in vocab:
      validation_data[idx_sample][idx_word] = 0 ## UNK for vocabs that are not in the Vocab.
    else:
      validation_data[idx_sample][idx_word] = vocab.index(word)    


for idx_sample, sample in enumerate(test_data):
  for idx_word, word in enumerate(sample):
    if word not in vocab:
      test_data[idx_sample][idx_word] = 0 ## UNK for vocabs that are not in the Vocab.
    else:
      test_data[idx_sample][idx_word] = vocab.index(word)    



In [ ]:
#### Train data is 2D array and Train label is 1D array
print("Sample 0 of train data : {}".format(train_data[0]))
print("Sample 0 of train label : {}".format(train_label[0]))

#### Validation data is 2D array and Validation label is 1D array

print("Sample 0 of validation data : {}".format(validation_data[0]))
print("Sample 0 of validation label : {}".format(validation_label[0]))

#### Test data is 2D array and Test label is 1D array

print("Sample 0 of test data : {}".format(test_data[0]))
print("Sample 0 of test label : {}".format(test_label[0]), end = "\n")


Sample 0 of train data : [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 5, 14, 15, 16, 17, 18, 19, 20, 12, 21, 22, 23, 24, 25, 15, 26, 27, 16, 28, 29, 30, 31, 0, 32, 33, 34, 12, 5, 35, 0, 36, 37, 38, 39, 40, 41, 42, 12, 5, 39, 43, 4, 44, 45, 46, 47, 48, 12, 49, 4, 50, 51, 26, 52, 2, 53, 54, 55, 56, 57, 58, 59, 60, 5, 15, 61, 4, 62, 44, 63, 64, 5, 65, 10, 66, 67, 12, 22, 68, 58, 69, 17, 70, 60, 71, 72, 4, 73, 74, 17, 75, 76, 12, 5, 77, 26, 78, 79, 33, 80, 81, 60, 82, 83, 12, 51, 26, 84, 85, 2, 17, 86, 87, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Sample 0 of train label : 0
Sample 0 of validation data : [3407, 57469, 43747, 21, 22, 26, 5, 2385, 113, 655, 89, 329, 10, 146, 189, 5, 2617, 1407, 12, 0, 8513, 4, 33, 176, 113, 735, 12, 22, 2385, 113, 187, 33, 11397, 200, 40974, 1008, 5, 408, 12, 1077, 135, 17, 1961, 6026, 33, 253, 145,

In [ ]:
with open("/content/drive/MyDrive/sentiment/vocab.txt", "wb") as fp:   
   pickle.dump(vocab, fp)

with open("/content/drive/MyDrive/sentiment/train_data_indexed.txt", "wb") as fp:   
   pickle.dump(train_data, fp)
with open("/content/drive/MyDrive/sentiment/validation_data_indexed.txt", "wb") as fp:   
   pickle.dump(validation_data, fp)
with open("/content/drive/MyDrive/sentiment/test_data_indexed.txt", "wb") as fp:   
   pickle.dump(test_data, fp)



with open("/content/drive/MyDrive/sentiment/train_lebel.txt", "wb") as fp:   
   pickle.dump(train_label, fp)
with open("/content/drive/MyDrive/sentiment/validation_label.txt", "wb") as fp:  
   pickle.dump(validation_label, fp)
with open("/content/drive/MyDrive/sentiment/test_label.txt", "wb") as fp:  
   pickle.dump(test_label, fp)

print("Done !!!")


Done !!!
